<a href="https://colab.research.google.com/github/SandroPaiva/projeto_chatbot_alura/blob/main/chat_atendimeto_construtora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instalação da biblioteca google-genai
!pip install -q google-genai


In [9]:
!pip install -q google-adk

In [2]:
#Importação do módulo 'os' que permite interagir com o sistema operacional, aqui é usado para
#criar váriáveis de ambiente
import os
from google.colab import userdata

#Variável de ambiente que armazenará a chave API para validação de credencial
#de forma segura.
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [3]:
#importação do módulo genai do pacote google, que faz parte do SDK Python do
#Goggle Generative AI e fornece ferramentas para interagir com os modelos de
#AI generativas do Google
from google import genai

#Criação de um Objeto Cliente que nada mais é que uma instância da classe Client do genai
cliente = genai.Client()

In [ ]:
#Aqui será listada a lista de modelos, será usada apenas para consunta para saber
#quais modelos disponiveis para uso.
for model in cliente.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-p

In [ ]:
import os
# Se estiver rodando no Google Colab e usando userdata para a chave da API
from google.colab import userdata
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import markdown
import json
import ast # Importado para possível uso em parsing mais robusto, embora vamos tentar json.loads primeiro
from google.adk.tools import google_search


# --- Configuração da chave da API ---
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

# Configuração da geração
generation_config = {
    "temperature": 0.7,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 2048,
}

# Configuração de segurança
"""
safety_settings = [
    {"category": HarmCategory.HARMON_CATEGORY_HARASSMENT, "threshold": HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE},
    {"category": HarmCategory.HARMON_CATEGORY_HATE_SPEECH, "threshold": HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE},
    {"category": HarmCategory.HARMON_CATEGORY_SEXUALLY_EXPLICIT, "threshold": HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE},
    {"category": HarmCategory.HARMON_CATEGORY_DANGEROUS_CONTENT, "threshold": HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE},
]
"""
# Inicialização do modelo Gemini
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config=generation_config,
    safety_settings=safety_settings
)

# Inicia uma nova sessão de chat
chat = model.start_chat(history=[])

# Contador de perguntas fora do contexto
contador_fora_contexto = 0
LIMITE_FORA_CONTEXTO = 3

# ---------------------------------------------------------------

"""
Classe usada para criar um catálogo fictício de empreendimentos imobiliários.
"""
class CatalogoEmpreendimentosFicticio:
    def __init__(self):
        self._empreendimentos = [
            {
                "id": 1,
                "nome": "Residencial Vista Alegre",
                "bairro": "Jabaquara",
                "tipo": "Apartamento",
                "status": "À venda",
                "detalhes": "3 quartos, 2 vagas, varanda gourmet. Lazer completo."
            },
            {
                "id": 2,
                "nome": "Edifício Central Park",
                "bairro": "Centro",
                "tipo": "Apartamento",
                "status": "À venda",
                "detalhes": "2 quartos, próximo ao metrô. Ideal para investimento."
            },
            {
                "id": 3,
                "nome": "Villa Harmonia",
                "bairro": "Moema",
                "tipo": "Casa de Condomínio",
                "status": "À venda",
                "detalhes": "4 suítes, piscina privativa, segurança 24h."
            },
            {
                "id": 4,
                "nome": "Studios Urban Living",
                "bairro": "Perdizes",
                "tipo": "Studio",
                "status": "À venda",
                "detalhes": "Moderno e compacto, com serviços pay-per-use."
            },
            {
                "id": 5,
                "nome": "Horizonte Azul Offices",
                "bairro": "Centro",
                "tipo": "Sala Comercial",
                "status": "Locação",
                "detalhes": "Salas comerciais modulares, excelente localização."
            },
            {
                "id": 6,
                "nome": "Residencial Pôr do Sol",
                "bairro": "Lapa",
                "tipo": "Apartamento",
                "status": "Vendido",
                "detalhes": "2 quartos, recém-reformado."
            },
             {
                "id": 7,
                "nome": "Campos de Aruanda",
                "bairro": "jabaquara",
                "tipo": "Casa",
                "status": "À Venda",
                "detalhes": "4 quartos, duas suítes, imovél novo."
            },
            {
                "id": 8,
                "nome": "Residencial Nosso Lar",
                "bairro": "jabaquara",
                "tipo": "Casa",
                "status": "À Venda",
                "detalhes": "3 quartos, garagem, imovél novo."
            }
        ]

    def buscar_por_bairro(self, nome_bairro):
        nome_bairro_lower = nome_bairro.lower()
        encontrados = []
        for emp in self._empreendimentos:
            if emp.get("bairro", "").lower() == nome_bairro_lower:
                encontrados.append(emp)
        return encontrados

    def buscar_por_tipo_e_status(self, tipo, status):
        tipo_lower = tipo.lower() if tipo else None
        status_lower = status.lower() if status else None
        encontrados = []
        for emp in self._empreendimentos:
            match_tipo = not tipo_lower or (emp.get("tipo", "").lower() == tipo_lower)
            match_status = not status_lower or (emp.get("status", "").lower() == status_lower)
            if match_tipo and match_status:
                encontrados.append(emp)
        return encontrados

    def listar_bairros_com_empreendimentos(self):
        bairros = set()
        for emp in self._empreendimentos:
            if emp.get("bairro"):
                bairros.add(emp["bairro"])
        return list(bairros)

# Instancia o catálogo fictício
catalogo = CatalogoEmpreendimentosFicticio()

def interpretar_pergunta(pergunta):
    """
    Utiliza o Gemini para interpretar a intenção do usuário e extrair informações relevantes.
    Retorna um dicionário com tipo, status e bairro se reconhecer a intenção de busca,
    caso contrário, retorna None. Mais robusto para parsing JSON.
    """
    prompt_interpretacao = (
        f"Interprete a seguinte pergunta de um usuário buscando imóveis e extraia as seguintes informações, se presentes:\n"
        f"- Tipo de imóvel (casa, apartamento, casa de condomínio, studio, sala comercial)\n"
        f"- Status do imóvel (à venda, para alugar, comprar)\n"
        f"- Nome do bairro\n\n"
        f"Formato de saída esperado:\n"
        f"Se a pergunta for sobre buscar imóveis, retorne APENAS um dicionário JSON com as chaves 'tipo', 'status' e 'bairro'. Se alguma informação não estiver presente na pergunta, o valor correspondente deve ser null. **USE APENAS ASPAS DUPLAS (\") para chaves e valores string.**\n" # Instrução adicional para aspas duplas
        f"Se a pergunta NÃO for sobre buscar imóveis, retorne APENAS a string literal 'OUTRO' (sem aspas extras, sem formatação).\n\n"
        f"Exemplo 1: 'Quero casas à venda no Jabaquara' -> {{'tipo': 'casa', 'status': 'à venda', 'bairro': 'Jabaquara'}}\n"
        f"Exemplo 2: 'Apartamentos para alugar em Moema' -> {{'tipo': 'apartamento', 'status': 'para alugar', 'bairro': 'Moema'}}\n"
        # ... (outros exemplos) ...
        f"Pergunta: '{pergunta}'\n\n"
        f"Resposta (APENAS JSON com aspas duplas ou 'OUTRO'):" # Reitera a instrução
    )
    try:
        response = model.generate_content(prompt_interpretacao)
        resposta_texto = response.text.strip()

        #print(f"DEBUG (Resposta bruta da interpretação): '{resposta_texto}'") # Debug raw text

        # Tenta limpar formatação de código markdown
        if resposta_texto.startswith('```'):
             resposta_texto = resposta_texto.strip('`')
             # Remove possível linguagem especificada no topo (ex: json)
             primeira_linha = resposta_texto.splitlines()[0]
             if not primeira_linha.strip().startswith('{'): # Se a primeira linha não é o JSON abrindo, tenta remover
                  resposta_texto = '\n'.join(resposta_texto.splitlines()[1:])
             resposta_texto = resposta_texto.strip()


        #print(f"DEBUG (Resposta após limpeza inicial): '{resposta_texto}'") # Debug after initial cleanup

        if resposta_texto.lower() == 'outro':
            return None
        else:
            try:
                # Tenta carregar como JSON padrão primeiro
                return json.loads(resposta_texto)
            except json.JSONDecodeError as json_e:
                ##print(f"DEBUG: JSONDecodeError padrão: {json_e}. Tentando corrigir aspas simples.") # Debug erro JSON padrão

                # Tenta corrigir aspas simples para duplas como uma tentativa de recuperação
                # CUIDADO: Isso é uma correção básica e pode não funcionar em todos os casos complexos.
                fixed_resposta_texto = resposta_texto.replace("'", '"')
                try:
                     # Tenta carregar o JSON com as aspas corrigidas
                     parsed_data = json.loads(fixed_resposta_texto)
                     ##print(f"DEBUG: JSON corrigido com sucesso. Resultado: {parsed_data}")
                     return parsed_data
                except json.JSONDecodeError as fixed_json_e:
                     ##print(f"DEBUG: JSONDecodeError mesmo após tentar corrigir aspas: {fixed_json_e}") # Debug erro na correção
                     ##print(f"DEBUG: Resposta do Gemini não é JSON válido (mesmo após tentar corrigir aspas): '{resposta_texto}'")
                     return None # Retorna None se ainda for inválido após a correção

    except Exception as e:
        ##print(f"DEBUG: Erro geral durante a interpretação pelo Gemini: {e}")
        return None


def generar_resposta_catalogo(empreendimentos, tipo=None, status=None, bairro=None):
    """
    Gera uma resposta formatada em Markdown com a lista de empreendimentos do CATÁLOGO.
    Retorna a string com a sintaxe Markdown.
    """
    if not empreendimentos:
        if bairro:
             bairros_no_catalogo_lower = [b.lower() for b in catalogo.listar_bairros_com_empreendimentos()]
             if bairro.lower() in bairros_no_catalogo_lower:
                 return f"Parece que não temos **{tipo.lower() if tipo else 'imóveis'}** para **{status.lower() if status else 'qualquer status'}** no bairro de **{bairro}** no nosso catálogo interno. 😔"
             else:
                 return f"Não encontramos empreendimentos no bairro de **{bairro}** no nosso catálogo interno. 😔"
        elif tipo or status:
            return f"Não encontramos **{tipo.lower() if tipo else 'imóveis'}** para **{status.lower() if status else 'qualquer status'}** no nosso catálogo interno. 😔"
        else:
             return "Não encontramos empreendimentos no nosso catálogo interno com os critérios especificados. 😔"


    resposta = ""
    if bairro:
        resposta += f"🏠✨ **Encontrei estes empreendimentos no bairro de {bairro} no nosso catálogo interno:**\n\n"
    elif tipo and status:
        resposta += f"🔑 Busca por **{tipo.capitalize()} para {status.capitalize()}** no nosso catálogo interno:\n\n"
    elif tipo:
        resposta += f"🏡 Busca por **{tipo.capitalize()}** no nosso catálogo interno:\n\n"
    else:
        resposta += "🔍 **Todos os Empreendimentos do nosso catálogo interno:**\n\n"

    for emp in empreendimentos:
        resposta += f"- **{emp.get('nome', 'Nome não disponível')}** ({emp.get('tipo', 'Tipo não disponível')})\n"
        resposta += f"  - Bairro: {emp.get('bairro', 'Não informado')}\n"
        resposta += f"  - Status: {emp.get('status', 'Não informado')}\n"
        resposta += f"  - Detalhes: {emp.get('detalhes', 'Nenhum detalhe adicional.')}\n\n"

    return resposta # Retorna a string Markdown

# Assumindo que 'google_search' é importado e possui um método 'search' que aceita 'queries=[query]'
# Certifique-se de que 'from google.adk.tools import google_search' está ativa no seu ambiente Colab.
def buscar_na_web(query):
    """
    Utiliza a ferramenta google_search para buscar informações.
    Assume que google_search(queries=[query]) retorna uma lista de resultados
    onde cada resultado tem, no mínimo, 'title', 'url', 'snippet' dentro de uma sublista 'results'.
    Retorna a string com a sintaxe Markdown dos resultados.
    """
    try:
        # --- !!! CHAMADA REAL PARA A FERRAMENTA google_search !!! ---
        # Passando a query como uma lista, conforme a documentação do tool code
        search_results_raw = google_search(queries=[query])
        # --- !!! FIM DA CHAMADA REAL !!! ---

        ##print(f"DEBUG: Resposta bruta do google_search: {search_results_raw}") # Debug raw search results

        # Verifica se a lista principal não está vazia e se há resultados dentro do primeiro item
        resultados_validos = []
        if search_results_raw and isinstance(search_results_raw, list) and len(search_results_raw) > 0:
            primeiro_resultado = search_results_raw[0]
            if isinstance(primeiro_resultado, dict) and 'results' in primeiro_resultado and isinstance(primeiro_resultado['results'], list):
                 resultados_validos = primeiro_resultado['results']

        if not resultados_validos:
             ##print(f"DEBUG: google_search retornou vazio ou sem resultados válidos para a query: '{query}'")
             return "🌐 Não encontrei resultados relevantes na web para sua consulta. Que tal tentar outra busca? 🤔"


        resposta = "🌐 **Encontrei algumas opções na web:**\n\n"
        for i, result in enumerate(resultados_validos[:5]): # Limita a 5 resultados válidos
            try:
                title = result.get('title', 'Sem título')
                url = result.get('url', '#')
                snippet = result.get('snippet', 'Sem descrição disponível.')

                # Formata como um link Markdown (clicável no Colab)
                resposta += f"**{i+1}. [{title}]({url})**\n"
                resposta += f"{snippet}\n\n"
            except Exception as e:
                ##print(f"DEBUG: Erro ao processar resultado individual da busca web: {e}")
                continue

        return resposta # Retorna a string Markdown

    except Exception as e:
        ##print(f"DEBUG: Ocorreu um erro ao realizar busca na web: {e}")
        return "❌ Ocorreu um erro inesperado ao tentar buscar na web. Por favor, tente novamente mais tarde. 🤔"


# Mensagem de boas vindas
print("🤖 Olá! Sou o assistente virtual da Construtora XYZ. Como posso te ajudar a encontrar seu imóvel dos sonhos hoje? ✨")

# Laço de repetição para conversar com o usuário
while True:
    pergunta_usuario = input("👤 Você: ")
    if pergunta_usuario.lower().strip() == 'fim':
        print("🤖 Especialista Python: Combinado! Até logo! 👋")
        break

    # Interpretando a pergunta do usuário
    interpretacao = interpretar_pergunta(pergunta_usuario)
    # O print de debug da interpretação está dentro da função interpretar_pergunta

    # --- LIDANDO COM PERGUNTAS FORA DE CONTEXTO OU SEGUIMENTOS ---
    # ESTE BLOCO FOI CORRIGIDO PARA GARANTIR O INCREMENTO CORRETO DO CONTADOR
    if interpretacao is None:

        prompt_classificar_seguimento = (
            f"O usuário acabou de fazer a seguinte pergunta: '{pergunta_usuario}'. "
            f"Considerando o histórico recente da conversa sobre busca de imóveis, "
            f"essa nova pergunta '{pergunta_usuario}' parece ser um seguimento ou "
            f"uma pergunta sobre os resultados da busca de imóveis anterior "
            f"(por exemplo, perguntando sobre um resultado exibido, pedindo mais detalhes "
            f"de um imóvel mencionado, ou refinando a busca anterior)? "
            f"Responda APENAS 'SIM' ou 'NAO' (sem pontuação ou formatação extra)."
        )
        try:
            resposta_classificacao = model.generate_content(prompt_classificar_seguimento)
            texto_classificacao = resposta_classificacao.text.strip().upper()

            #print(f"DEBUG (Classificação follow-up): '{texto_classificacao}'")

            if texto_classificacao == 'SIM':
                # ✅ É um seguimento sobre a busca de imóveis. NÃO INCREMENTAR O CONTADOR AQUI.
                ##print("DEBUG: Pergunta classificada como seguimento (SIM). Contador NÃO foi incrementado.") # Debug

                # Permite que o Gemini responda conversacionalmente sobre o resultado/seguimento.
                try:
                    resposta_gemini_seguimento = chat.send_message(pergunta_usuario)
                    print(f"🤖 Chatbot: {resposta_gemini_seguimento.text} Deseja procurar por casas ou apartamentos? 🤔")

                    chat.history.append({"role": "user", "parts": [{"text": pergunta_usuario}]})
                    chat.history.append({"role": "model", "parts": [{"text": resposta_gemini_seguimento.text}]})
                except Exception as e:
                     ##print(f"DEBUG: Erro ao gerar resposta de seguimento pelo Gemini: {e}")
                     print("🤖 Chatbot: Desculpe, tive um problema para responder sobre isso. Deseja procurar por casas ou apartamentos? 🤔")
                     chat.history.append({"role": "user", "parts": [{"text": pergunta_usuario}]})
                     chat.history.append({"role": "model", "parts": [{"text": "Desculpe, tive um problema para responder sobre isso."}]})

                continue # Volta para o início do loop

            else: # texto_classificacao é 'NAO' ou algo inesperado
                # ❌ Não é um seguimento ou a classificação falhou de outra forma. TRATAR COMO FORA DE CONTEXTO.
                contador_fora_contexto += 1 # <<< CONTADOR INCREMENTA SOMENTE NESTE RAMO
                ##print(f"DEBUG: Pergunta classificada como fora de contexto (NAO). Contador: {contador_fora_contexto}")

                if contador_fora_contexto > LIMITE_FORA_CONTEXTO:
                    print("🤖 Chatbot: Entendido. Parece que no momento não há mais perguntas sobre imóveis.")
                    try:
                         resposta_gemini_despedida = chat.send_message("Como sou especializado em imóveis, não consigo te ajudar com outros assuntos agora. Agradeço o seu contato. Se tiver alguma dúvida sobre casas ou apartamentos no futuro, pode me chamar novamente. Até logo! 👋")
                         print(f"🤖 Chatbot: {resposta_gemini_despedida.text}")
                    except Exception as e:
                         ##print(f"DEBUG: Erro ao gerar despedida pelo Gemini: {e}")
                         print("🤖 Chatbot: Foi um prazer conversar com você. Se tiver alguma dúvida sobre imóveis no futuro, pode me chamar. Até logo! 👋")

                    break # Sai do loop

                else:
                    # Permite que o Gemini responda livremente à pergunta fora de contexto
                    try:
                        resposta_gemini = chat.send_message(pergunta_usuario)
                        print(f"🤖 Chatbot: {resposta_gemini.text} Deseja procurar por casas ou apartamentos? 🤔")

                        chat.history.append({"role": "user", "parts": [{"text": pergunta_usuario}]})
                        chat.history.append({"role": "model", "parts": [{"text": resposta_gemini.text}]})
                    except Exception as e:
                         ##print(f"DEBUG: Erro ao gerar resposta fora do contexto pelo Gemini: {e}")
                         print("🤖 Chatbot: Desculpe, tive um problema para responder. Deseja procurar por casas ou apartamentos? 🤔")
                         chat.history.append({"role": "user", "parts": [{"text": pergunta_usuario}]})
                         chat.history.append({"role": "model", "parts": [{"text": "Desculpe, tive um problema para responder."}]})

                    continue # Volta para o início do loop


        except Exception as e:
            # --- SE OCORRER UM ERRO NA CHAMADA DE CLASSIFICAÇÃO ---
            ##print(f"DEBUG: Erro durante a classificação de seguimento pelo Gemini: {e}")
            # Em caso de erro na classificação, TRATAR COMO FORA DE CONTEXTO por segurança.
            contador_fora_contexto += 1 # <<< CONTADOR INCREMENTA TAMBÉM NESTE RAMO EM CASO DE ERRO
            ##print(f"DEBUG: Erro na classificação. Tratando como fora de contexto. Contador: {contador_fora_contexto}")

            if contador_fora_contexto > LIMITE_FORA_CONTEXTO:
                print("🤖 Chatbot: Entendido. Parece que no momento no há mais perguntas sobre imóveis.")
                try:
                     resposta_gemini_despedida = chat.send_message("Como sou especializado em imóveis, não consigo te ajudar com outros assuntos agora. Agradeço o seu contato. Se tiver alguma dúvida sobre casas ou apartamentos no futuro, pode me chamar novamente. Até logo! 👋")
                     print(f"🤖 Chatbot: {resposta_gemini_despedida.text}")
                except:
                     print("🤖 Chatbot: Foi um prazer conversar com você. Se tiver alguma dúvida sobre imóveis no futuro, pode me chamar. Até logo! 👋")

                break
            else:
                 try:
                     resposta_gemini = chat.send_message(pergunta_usuario)
                     print(f"🤖 Chatbot: {resposta_gemini.text} Deseja procurar por casas ou apartamentos? 🤔")
                     chat.history.append({"role": "user", "parts": [{"text": pergunta_usuario}]})
                     chat.history.append({"role": "model", "parts": [{"text": resposta_gemini.text}]})
                 except:
                      print("🤖 Chatbot: Desculpe, tive um problema para responder. Deseja procurar por casas ou apartamentos? 🤔")
                      chat.history.append({"role": "user", "parts": [{"text": pergunta_usuario}]})
                      chat.history.append({"role": "model", "parts": [{"text": "Desculpe, tive um problema para responder."}]})
                 continue


    # --- Se a interpretação não foi None (intenção de busca) ---
    else:
        # ESTE BLOCO PERMANECE PRATICAMENTE IGUAL NA LÓGICA DE BUSCA, APENAS COM DEBUGs
        contador_fora_contexto = 0 # Reseta o contador (isso já estava correto)

        # --- Mapeamento de Status ---
        status_imovel_raw = interpretacao.get('status')
        status_imovel_mapped = None
        if status_imovel_raw:
            status_imovel_lower = status_imovel_raw.lower()
            if 'venda' in status_imovel_lower or 'comprar' in status_imovel_lower:
                status_imovel_mapped = 'À venda'
            elif 'alugar' in status_imovel_lower or 'locacao' in status_imovel_lower:
                status_imovel_mapped = 'Locação'

        tipo_imovel = interpretacao.get('tipo')
        status_imovel_catalogo = status_imovel_mapped
        bairro_imovel = interpretacao.get('bairro')

        ##print(f"DEBUG: Interpretação VÁLIDA - Tipo: {tipo_imovel}, Status (Raw): {status_imovel_raw}, Status (Mapped): {status_imovel_catalogo}, Bairro: {bairro_imovel}") # DEBUG

        empreendimentos_encontrados = []
        procurar_na_web = False
        web_search_query = ""
        resposta_final_chatbot = ""

        # --- Lógica de Busca (Catálogo vs. Web) ---
        bairros_no_catalogo_lower = [b.lower() for b in catalogo.listar_bairros_com_empreendimentos()]
        bairro_existe_no_catalogo = bairro_imovel and (bairro_imovel.lower() in bairros_no_catalogo_lower)

        ##print(f"DEBUG: Bairro '{bairro_imovel}' existe no catálogo? {bairro_existe_no_catalogo}") # DEBUG

        # --- DECIDE SEARCH STRATEGY ---
        should_try_catalog_first = False

        if bairro_imovel and not bairro_existe_no_catalogo:
            # Caso 2: Bairro solicitado, mas NÃO existe no catálogo. Buscar na web diretamente.
            ##print("DEBUG: Caso 2: Bairro não existe no catálogo. Definindo procurar_na_web = True.") # DEBUG
            procurar_na_web = True
            query_parts = []
            if tipo_imovel: query_parts.append(tipo_imovel)
            if status_imovel_raw: query_parts.append(f"para {status_imovel_raw}")
            if bairro_imovel: query_parts.append(f"no bairro {bairro_imovel}")
            web_search_query = " ".join(query_parts) + " imóveis" if query_parts else "imóveis"
            if not web_search_query.strip(): web_search_query = pergunta_usuario

        elif (tipo_imovel is not None or status_imovel_catalogo is not None) or (bairro_imovel is not None and bairro_existe_no_catalogo):
             # Caso 3: Tipo/Status solicitado (sem bairro ou bairro existe)
             # Caso 1b (parte 1): Bairro solicitado E existe
             # Nestes casos, devemos TENTAR o catálogo primeiro.
             should_try_catalog_first = True
             ##print("DEBUG: Tipo/Status solicitado ou Bairro existente. Tentando catálogo primeiro.") # DEBUG


        if should_try_catalog_first:
            ##print("DEBUG: Tentando buscar no catálogo...") # DEBUG
            if bairro_imovel is not None and bairro_existe_no_catalogo:
                 # Caso 1b (parte 1): Bairro solicitado E existe. Buscar critérios específicos dentro do bairro.
                 empreendimentos_no_bairro = catalogo.buscar_por_bairro(bairro_imovel)
                 ##print(f"DEBUG: Encontrados {len(empreendimentos_no_bairro)} imóveis no bairro {bairro_imovel} no catálogo antes de filtrar.") # DEBUG

                 if tipo_imovel is not None or status_imovel_catalogo is not None:
                      # Filtrar por tipo/status
                      for emp in empreendimentos_no_bairro:
                           match_type = not tipo_imovel or (emp.get('tipo', '').lower() == tipo_imovel.lower() if emp.get('tipo') else False)
                           match_status = not status_imovel_catalogo or (emp.get('status', '').lower() == status_imovel_catalogo.lower() if emp.get('status') else False)
                           if match_type and match_status:
                                empreendimentos_encontrados.append(emp)
                      ##print(f"DEBUG: Encontrados {len(empreendimentos_encontrados)} imóveis no catálogo após filtrar por Tipo/Status no bairro.") # DEBUG

                      if not empreendimentos_encontrados:
                           # Caso 1b (parte 2): Bairro existe, mas a combinação Tipo/Status NÃO foi encontrada no catálogo. Buscar na web.
                           ##print("DEBUG: Caso 1b (parte 2): Bairro existe, mas combinação Tipo/Status não encontrada no catálogo. Definindo procurar_na_web = True.") # DEBUG
                           procurar_na_web = True
                           query_parts = []
                           if tipo_imovel: query_parts.append(tipo_imovel)
                           if status_imovel_raw: query_parts.append(f"para {status_imovel_raw}")
                           if bairro_imovel: query_parts.append(f"no bairro {bairro_imovel}")
                           web_search_query = " ".join(query_parts) + " imóveis" if query_parts else "imóveis"
                           if not web_search_query.strip(): web_search_query = pergunta_usuario

                      # Else: empreendimentos_encontrados is not empty -> encontrados no catálogo, procurar_na_web permanece False

                 else:
                      # Caso 1b (parte 3): Apenas Bairro foi solicitado (e existe). Listar todos nesse bairro do catálogo.
                      ##print("DEBUG: Caso 1b (parte 3): Apenas Bairro especificado (e existe). Listando todos do catálogo nesse bairro.") # DEBUG
                      empreendimentos_encontrados = empreendimentos_no_bairro # Encontrados no catálogo, procurar_na_web permanece False

            elif tipo_imovel is not None or status_imovel_catalogo is not None:
                 # Caso 3: Tipo/Status solicitado, mas SEM bairro. Tenta catálogo primeiro.
                 ##print("DEBUG: Caso 3: Buscando por Tipo/Status no catálogo (sem bairro).") # DEBUG
                 empreendimentos_encontrados = catalogo.buscar_por_tipo_e_status(tipo_imovel, status_imovel_catalogo) # Usa o status mapeado
                 ##print(f"DEBUG: Encontrados {len(empreendimentos_encontrados)} imóveis no catálogo para Tipo/Status (sem bairro).") # DEBUG


                 if not empreendimentos_encontrados:
                      # Não encontrou no catálogo. Buscar na web.
                      ##print("DEBUG: Caso 3: Tipo/Status não encontrados no catálogo. Definindo procurar_na_web = True.") # DEBUG
                      procurar_na_web = True
                      query_parts = []
                      if tipo_imovel: query_parts.append(tipo_imovel)
                      if status_imovel_raw: query_parts.append(f"para {status_imovel_raw}")
                      web_search_query = " ".join(query_parts) + " imóveis" if query_parts else "imóveis"
                      if not web_search_query.strip(): web_search_query = pergunta_usuario
                 # Else: empreendimentos_encontrados is not empty -> encontrados no catálogo, procurar_na_web permanece False

        else: # Nenhuma condição de busca específica foi atendida para tentar o catálogo primeiro
            # Caso 4: Nenhuma informação específica extraída (e não foi 'OUTRO'). Listar todos do catálogo.
            # Isso acontece se tipo_imovel, status_imovel_catalogo, E bairro_imovel são todos None.
            ##print("DEBUG: Caso 4: Sem critérios específicos. Listando todos do catálogo. Não buscando na web.") # DEBUG
            empreendimentos_encontrados = catalogo._empreendimentos
            procurar_na_web = False


        # --- Gerar Resposta Final ---
        ##print(f"DEBUG: Valor final de procurar_na_web antes da chamada da função de resposta: {procurar_na_web}") # DEBUG
        if procurar_na_web:
            ##print(f"DEBUG: Buscando na web com a query: '{web_search_query}'") # DEBUG
            resposta_final_chatbot = buscar_na_web(web_search_query)
        else:
            ##print("DEBUG: Gerando resposta do catálogo.") # DEBUG
            resposta_final_chatbot = generar_resposta_catalogo(empreendimentos_encontrados, tipo=tipo_imovel, status=status_imovel_catalogo, bairro=bairro_imovel)

        # Adiciona a pergunta persistente no final de TODAS as respostas de busca
        resposta_final_chatbot += " Deseja procurar por casas ou apartamentos? 🤔"

        # --- Exibir Resposta e Atualizar Histórico ---
        print(f"🤖 Chatbot: {resposta_final_chatbot}")

        chat.history.append({"role": "user", "parts": [{"text": pergunta_usuario}]})
        chat.history.append({"role": "model", "parts": [{"text": resposta_final_chatbot}]})


# --- FIM DA LÓGICA DE INTERAÇÃO ---